In [70]:
!npm install localtunnel


up to date, audited 23 packages in 911ms

3 packages are looking for funding
  run `npm fund` for details

2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [71]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelos de machine learning
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configuración de página
st.set_page_config(
    page_title="Análisis de Girona Airbnb",
    page_icon="🏰",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Combina ambos CSS en una sola definición
st.markdown("""
<style>
    .stApp {
        background-image: url("https://images.unsplash.com/photo-1553826059-7a090c4bd362");
        background-size: cover;
        background-position: center;
        background-attachment: fixed;
        min-height: 100vh;
    }
    
    /* Fondo semitransparente para el contenido */
    .main .block-container {
        background-color: rgba(255, 255, 255, 0.93) !important;
        border-radius: 15px;
        padding: 3rem !important;
        box-shadow: 0 6px 12px rgba(0,0,0,0.15);
        margin-top: 2rem;
        margin-bottom: 2rem;
    }
    
    /* Ajustes para títulos */
    h1, h2, h3 {
        color: #DA291C !important;
        text-shadow: 1px 1px 2px rgba(255,255,255,0.7);
    }
    
    /* Sidebar y componentes */
    .css-1d391kg {
        background-color: rgba(218, 41, 28, 0.9) !important;
        color: white !important;
    }
    .st-bb, .st-at {
        background-color: #FFC72C !important;
        color: #333 !important;
    }
    .st-bq {
        border-color: #FFC72C !important;
    }
</style>
""", unsafe_allow_html=True)

# Cargar datos con cache
@st.cache_resource
def load_data():
    df = pd.read_csv("Girona_limpio.csv")
    df['price_high'] = (df['price'] > df['price'].mean()).astype(int)
    numeric_df = df.select_dtypes(['float','int'])
    numeric_cols = numeric_df.columns
    text_df = df.select_dtypes(['object'])
    text_cols = text_df.columns
    unique_categories_room_type = df['room_type'].unique()
    return df, numeric_cols, text_cols, unique_categories_room_type, numeric_df

# Cargamos los datos
df, numeric_cols, text_cols, unique_categories_room_type, numeric_df = load_data()

# Sidebar - Configuración general
with st.sidebar:
    # Asegúrate de que la imagen existe en tu directorio
    try:
        st.image("Girona-City-view.jpg", 
                 width=300, 
                 caption="Girona - La Perla del Ter")
    except FileNotFoundError:
        st.image("https://images.unsplash.com/photo-1553826059-7a090c4bd362",
                 width=300,
                 caption="Girona - Vista panorámica")
    
    st.title("🏰 Girona Airbnb Dashboard")
    st.markdown("""
    **Dashboard interactivo** para analizar los datos de alojamientos Airbnb en Girona.
    """)
    
    View = st.selectbox(label="Vista", options=[
        "Exploración de Datos", 
        "Análisis por Categoría", 
        "Gráficos de Pastel", 
        "Análisis de Dispersión",
        "Gráficos de Barras", 
        "Matriz de Correlación",
        "Regresión Lineal",
        "Regresión Logística",
        "Regresión Lineal Múltiple"
    ])
    
    st.markdown("---")
    st.markdown("**Configuración de visualización**")
    st.markdown("""
    *Datos de alojamientos turísticos en la ciudad de Girona*
    """)

# VISTA 1: Exploración de Datos
if View == "Exploración de Datos":
    st.title("📊 Exploración de Datos - Airbnb Girona")
    st.markdown("""
    Explora el dataset completo de propiedades de Airbnb en **Girona**, ciudad conocida por su patrimonio histórico y cultural.
    """)
    
    with st.expander("ℹ Información sobre los datos", expanded=False):
        st.write("""
        - **Dataset**: Listados de Airbnb en Girona
        - **Fuente**: Datos públicos de Airbnb
        - **Periodo**: Datos actualizados
        - **Características**: Precios, tipos de alojamiento, ubicaciones, reseñas
        """)
    
    col1, col2 = st.columns([3, 1])
    
    with col1:
        st.subheader("Dataset completo")
        check_box = st.checkbox(label="Mostrar Dataset completo", value=False)
        
        if check_box:
            st.dataframe(df, use_container_width=True, height=400)
    
    with col2:
        st.subheader("Estadísticas clave")
        st.metric("Total de propiedades", len(df), help="Número total de alojamientos listados")
        st.metric("Tipos de habitación", len(unique_categories_room_type), help="Variedad de tipos de alojamiento")
        st.metric("Precio promedio", f"€{df['price'].mean():.2f}", help="Precio medio por noche")
    
    st.markdown("---")
    
    st.subheader("Resumen estadístico")
    st.dataframe(df.describe(), use_container_width=True)

# VISTA 2: Análisis por Categoría
elif View == "Análisis por Categoría":
    st.title("📈 Análisis por Tipo de Alojamiento")
    st.markdown("""
    Comparativa de variables numéricas según el tipo de alojamiento en Girona.
    """)
    
    with st.container():
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("Configuración")
            numerics_vars_selected = st.multiselect(
                label="Variables a analizar", 
                options=numeric_cols,
                default=["price", "number_of_reviews"],
                help="Selecciona las métricas a comparar"
            )
            
        with col2:
            st.subheader("Filtros")
            category_selected = st.selectbox(
                label="Tipo de habitación", 
                options=unique_categories_room_type,
                help="Filtra por tipo de alojamiento"
            )
    
    if numerics_vars_selected:
        data = df[df['room_type'] == category_selected]
        data_features = data[numerics_vars_selected]
        
        tab1, tab2 = st.tabs(["Gráfico de Líneas", "Datos"])
        
        with tab1:
            fig = px.line(
                data_frame=data_features, 
                x=data_features.index, 
                y=numerics_vars_selected, 
                title=f'Análisis para: {category_selected} en Girona',
                width=1600, 
                height=600,
                color_discrete_sequence=["#DA291C", "#FFC72C", "#5B6770"]  # Colores de Girona
            )
            st.plotly_chart(fig, use_container_width=True)
        
        with tab2:
            st.dataframe(data_features, use_container_width=True)

# VISTA 3: Gráficos de Pastel
elif View == "Gráficos de Pastel":
    st.title("🥧 Distribución por Categorías")
    st.markdown("""
    Visualización de proporciones según categorías de alojamiento en Girona.
    """)
    
    col1, col2 = st.columns(2)
    
    with col1:
        Variable_cat = st.selectbox(
            label="Variable Categórica", 
            options=text_cols,
            index=0,
            help="Selecciona la categoría a analizar"
        )
    
    with col2:
        Variable_num = st.selectbox(
            label="Variable Numérica", 
            options=numeric_cols,
            index=0,
            help="Selecciona la métrica a distribuir"
        )
    
    fig = px.pie(
        data_frame=df, 
        names=df[Variable_cat], 
        values=df[Variable_num], 
        title=f'Distribución de {Variable_num} por {Variable_cat} en Girona',
        hole=0.3,
        color_discrete_sequence=px.colors.sequential.RdBu
    )
    
    st.plotly_chart(fig, use_container_width=True)

# VISTA 4: Análisis de Dispersión
elif View == "Análisis de Dispersión":
    st.title("🔍 Relación entre Variables")
    st.markdown("""
    Explora la relación entre diferentes variables del mercado de Airbnb en Girona.
    """)
    
    with st.expander("Configuración", expanded=True):
        col1, col2 = st.columns(2)
        
        with col1:
            x_selected = st.selectbox(
                label="Variable X", 
                options=numeric_cols,
                index=0,
                help="Variable para el eje horizontal"
            )
        
        with col2:
            y_selected = st.selectbox(
                label="Variable Y", 
                options=numeric_cols,
                index=1,
                help="Variable para el eje vertical"
            )
    
    columnas_hover = ['neighbourhood', 'room_type', 'price']
    columnas_hover = [col for col in columnas_hover if col in df.columns]
    
    fig = px.scatter(
        data_frame=df,
        x=x_selected, 
        y=y_selected, 
        title=f'Relación entre {x_selected} y {y_selected} en Girona',
        color='room_type',
        hover_data=columnas_hover,
        width=1000,
        height=600,
        color_discrete_map={
            'Entire home/apt': '#DA291C',
            'Private room': '#FFC72C',
            'Shared room': '#5B6770'
        }
    )
    
    st.plotly_chart(fig, use_container_width=True)

# VISTA 5: Gráficos de Barras
elif View == "Gráficos de Barras":
    st.title("📊 Comparativa por Categorías")
    st.markdown("""
    Comparación de métricas según diferentes categorías de alojamiento.
    """)
    
    col1, col2 = st.columns(2)
    
    with col1:
        Variable_cat = st.selectbox(
            label="Variable Categórica", 
            options=text_cols,
            index=0
        )
    
    with col2:
        Variable_num = st.selectbox(
            label="Variable Numérica", 
            options=numeric_cols,
            index=0
        )
    
    fig = px.bar(
        data_frame=df, 
        x=df[Variable_cat], 
        y=df[Variable_num], 
        title=f'{Variable_num} por {Variable_cat} en Girona',
        color=df[Variable_cat],
        text_auto=True,
        color_discrete_sequence=["#DA291C", "#FFC72C", "#5B6770"]
    )
    fig.update_layout(showlegend=False)
    
    st.plotly_chart(fig, use_container_width=True)

# VISTA 6: Matriz de Correlación
elif View == "Matriz de Correlación":
    st.title("🔄 Relaciones entre Variables")
    st.markdown("""
    Matriz de correlación que muestra las relaciones entre variables numéricas del dataset.
    """)
    
    selected_vars = st.multiselect(
        "Selecciona variables para el análisis",
        options=numeric_cols,
        default=numeric_cols[:5],
        help="Elige al menos 2 variables para comparar"
    )
    
    if len(selected_vars) >= 2:
        corr_matrix = df[selected_vars].corr()
        
        fig = px.imshow(
            corr_matrix,
            text_auto=True,
            aspect="auto",
            color_continuous_scale='RdBu',
            title='Matriz de Correlación - Girona Airbnb',
            labels=dict(color="Correlación")
        )
        
        st.plotly_chart(fig, use_container_width=True)
        
        with st.expander("Interpretación"):
            st.write("""
            - **1**: Correlación positiva perfecta
            - **-1**: Correlación negativa perfecta
            - **0**: No hay correlación
            """)
    else:
        st.warning("Se necesitan al menos 2 variables para generar el heatmap")

# VISTA 7: Regresión Lineal
elif View == "Regresión Lineal":
    st.title("📈 Modelo Predictivo Simple")
    st.markdown("""
    Modelo de regresión lineal simple para predecir relaciones entre variables en el mercado de Girona.
    """)
    
    col1, col2 = st.columns(2)
    
    with col1:
        x_var = st.selectbox(
            "Variable Independiente (X)",
            options=numeric_cols,
            index=0
        )
    
    with col2:
        y_var = st.selectbox(
            "Variable Dependiente (Y)",
            options=numeric_cols,
            index=1
        )
    
    # Filtrar valores NaN
    df_clean = df[[x_var, y_var]].dropna()
    X = df_clean[x_var].values
    y = df_clean[y_var].values
    
    if len(X) > 0 and len(y) > 0:
        # Calcular regresión lineal
        n = len(X)
        x_mean, y_mean = np.mean(X), np.mean(y)
        xy_cov = np.sum((X - x_mean) * (y - y_mean))
        x_var_value = np.sum((X - x_mean)**2)
        
        coef = xy_cov / x_var_value
        intercept = y_mean - (coef * x_mean)
        
        x_range = np.linspace(np.min(X), np.max(X), 100)
        y_pred = intercept + coef * x_range
        
        ss_tot = np.sum((y - y_mean)**2)
        ss_res = np.sum((y - (intercept + coef * X))**2)
        r2 = 1 - (ss_res / ss_tot)
        
        # Mostrar métricas
        col1, col2, col3 = st.columns(3)
        col1.metric("R² (Coef. Determinación)", f"{r2:.4f}")
        col2.metric("Intercepto", f"{intercept:.4f}")
        col3.metric("Coeficiente (Pendiente)", f"{coef:.4f}")
        
        # Crear figura
        fig, ax = plt.subplots(figsize=(10, 6))
        
        sns.scatterplot(x=X, y=y, ax=ax, alpha=0.6, color='#DA291C', label='Datos reales')
        sns.lineplot(x=x_range, y=y_pred, ax=ax, color='#FFC72C', 
                    linewidth=2.5, label='Línea de regresión')
        
        ax.set_title(f'Regresión Lineal en Girona: {y_var} ~ {x_var}')
        ax.set_xlabel(x_var)
        ax.set_ylabel(y_var)
        ax.legend()
        ax.grid(True)
        
        st.pyplot(fig)
        
        st.markdown(f"**Ecuación del modelo:** {y_var} = {intercept:.4f} + {coef:.4f} * {x_var}")
        
        with st.expander("Interpretación del modelo"):
            st.write(f"""
            - **R² (Coeficiente de Determinación)**: {r2:.4f}  
              Indica que el {r2*100:.2f}% de la variabilidad en {y_var} es explicada por {x_var}.
            
            - **Pendiente (Coeficiente)**: {coef:.4f}  
              Por cada unidad que aumenta {x_var}, {y_var} {'aumenta' if coef > 0 else 'disminuye'} en {abs(coef):.4f} unidades.
            
            - **Intercepto**: {intercept:.4f}  
              Valor esperado de {y_var} cuando {x_var} es 0.
            """)
    else:
        st.warning("No hay suficientes datos para realizar la regresión con las variables seleccionadas.")

# VISTA 8: Regresión Logística
elif View == "Regresión Logística":
    st.title("📊 Modelo de Clasificación")
    st.markdown("""
    Modelo de regresión logística para predecir si el precio de un alojamiento en Girona está por encima de la media.
    """)
    
    st.subheader("Selección de variables")
    features = st.multiselect(
        "Selecciona las variables predictoras",
        options=numeric_cols,
        default=['bedrooms', 'bathrooms', 'accommodates'],
        help="Variables que influyen en el precio"
    )
    
    if len(features) >= 1:
        # Preparar datos
        df_clean = df[features + ['price_high']].dropna()
        X = df_clean[features]
        y = df_clean['price_high']
        
        # Dividir en train y test
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42
        )
        
        # Estandarizar datos
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Entrenar modelo
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train_scaled, y_train)
        
        # Predecir
        y_pred = model.predict(X_test_scaled)
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        
        # Métricas
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        # Mostrar resultados
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("Rendimiento del Modelo")
            st.metric("Precisión (Accuracy)", f"{accuracy:.2f}")
            st.metric("Precisión (Precision - Clase 1)", f"{report['1']['precision']:.2f}")
            st.metric("Recall (Clase 1)", f"{report['1']['recall']:.2f}")
            
        with col2:
            st.subheader("Matriz de Confusión")
            fig = px.imshow(
                conf_matrix,
                labels=dict(x="Predicho", y="Real", color="Cantidad"),
                x=['Precio Bajo (0)', 'Precio Alto (1)'],
                y=['Precio Bajo (0)', 'Precio Alto (1)'],
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu'
            )
            st.plotly_chart(fig, use_container_width=True)
        
        # Mostrar coeficientes
        st.subheader("Factores que influyen en el precio")
        coef_df = pd.DataFrame({
            'Variable': features,
            'Coeficiente': model.coef_[0],
            'Impacto': ['Positivo' if x > 0 else 'Negativo' for x in model.coef_[0]]
        }).sort_values('Coeficiente', ascending=False)
        
        st.dataframe(coef_df.style.applymap(
            lambda x: 'color: #DA291C' if x == 'Positivo' else 'color: #5B6770',
            subset=['Impacto']
        ), use_container_width=True)
        
        # Interpretación
        with st.expander("Interpretación del Modelo"):
            st.write("""
            - **Coeficientes Positivos**: Variables que aumentan la probabilidad de que el precio esté por encima de la media.
            - **Coeficientes Negativos**: Variables que disminuyen esta probabilidad.
            - **Matriz de Confusión**: Muestra los aciertos (diagonal) y errores del modelo.
            """)
    else:
        st.warning("Selecciona al menos una variable predictora")

# VISTA 9: Regresión Lineal Múltiple
elif View == "Regresión Lineal Múltiple":
    st.title("📊 Modelo Predictivo Avanzado")
    st.markdown("""
    Modelo de regresión lineal múltiple para predecir variables usando múltiples predictores.
    """)
    
    if len(numeric_cols) >= 2:
        X_cols = st.multiselect("Variables Independientes (X)", 
                               options=numeric_cols, 
                               default=numeric_cols[:2],
                               help="Selecciona las variables predictoras")
        y_col = st.selectbox("Variable Dependiente (y)", 
                            options=numeric_cols, 
                            index=1,
                            help="Variable a predecir")
        
        if len(X_cols) >= 1:
            temp_df = df[X_cols + [y_col]].dropna()
            X = temp_df[X_cols].values
            y = temp_df[y_col].values
            
            if len(X) > 0 and len(y) > 0:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                model = LinearRegression()
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                st.subheader("Factores del Modelo")
                coef_df = pd.DataFrame({
                    'Variable': X_cols + ['Intercepto'],
                    'Coeficiente': list(model.coef_) + [model.intercept_]
                })
                st.dataframe(coef_df.style.format("{:.4f}"), hide_index=True)
                
                fig = px.scatter(
                    x=y_test, 
                    y=y_pred, 
                    title="Valores Reales vs Predicciones en Girona",
                    labels={'x': 'Valores Reales', 'y': 'Predicciones'},
                    color_discrete_sequence=["#DA291C"],
                    trendline_color_override="#FFC72C"
                )
                fig.add_shape(
                    type="line", 
                    x0=min(y_test), 
                    y0=min(y_test),
                    x1=max(y_test), 
                    y1=max(y_test),
                    line=dict(color="#5B6770", dash="dash", width=2)
                )
                st.plotly_chart(fig, use_container_width=True)
                
                with st.expander("Resultados detallados"):
                    st.subheader("Comparación de predicciones")
                    pred_df = pd.DataFrame({
                        'Real': y_test[:20],
                        'Predicción': y_pred[:20],
                        'Diferencia': abs(y_test[:20] - y_pred[:20])
                    })
                    st.dataframe(pred_df.style.format("{:.2f}").background_gradient(
                        cmap="Reds", subset=["Diferencia"]
                    ))
            else:
                st.warning("No se pudo realizar la regresión debido a datos faltantes.")
        else:
            st.warning("Selecciona al menos una variable independiente.")
    else:
        st.warning("Se necesitan al menos dos variables numéricas para realizar la regresión.")

Overwriting app.py
